<a href="https://colab.research.google.com/github/alicia-tsai/privacy-open-sourced-av-data/blob/main/BDD_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Complete code to run on BDD images

In [ ]:
!pip install -q google-cloud-vision
import io
import os
from os.path import isfile, join
from collections import defaultdict
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab import files
from google.oauth2 import service_account
from google.cloud import vision
from google.colab import drive

     |████████████████████████████████| 440kB 6.5MB/s 


In [ ]:
# Remeber to get APIKEY

# modify this to suit your drive directory.
basedir = "/content/drive/My Drive/basedir"
imagedir = os.path.join(basedir, "images")   # directory to images

In [ ]:
import getpass

def authorize_app():
    # provide API key
    print("Please provide API Key")
    APIKEY = getpass.getpass()

    # provide application credentials
    print("Please upload Google application credentials")
    credential_json = files.upload()
    GOOGLE_APPLICATION_CREDENTIALS = os.path.join(os.getcwd(), "GOOGLE_APPLICATION_CREDENTIALS.json")

In [ ]:
authorize_app()
drive.mount('/content/drive') # mount google drive
IMAGES = [f for f in os.listdir(imagedir) if isfile(join(imagedir, f))]

Please provide API Key
··········
Please upload Google application credentials


Saving GOOGLE_APPLICATION_CREDENTIALS.json to GOOGLE_APPLICATION_CREDENTIALS.json
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
API_results = defaultdict()
total = len(IMAGES)

for i, IMAGE in enumerate(IMAGES):
    if i % 500 == 0:
      print("processed {}/{} images".format(i+1, total))
    
    with io.open(join(imagedir, IMAGE), 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.landmark_detection(image=image)
    landmarks = response.landmark_annotations
    if not landmarks:
        landmarks = None

    response = client.logo_detection(image=image)
    logos = response.logo_annotations
    if not logos:
        logo = None

    response = client.text_detection(image=image)
    texts = response.text_annotations
    if not texts:
        text = None

    API_results[IMAGE] = {}
    API_results[IMAGE]["landmarks"] = landmarks
    API_results[IMAGE]["logos"] = logos
    API_results[IMAGE]["texts"] = texts

processed 1/6369 images
processed 501/6369 images
processed 1001/6369 images
processed 1501/6369 images
processed 2001/6369 images
processed 2501/6369 images
processed 3001/6369 images
processed 3501/6369 images
processed 4001/6369 images
processed 4501/6369 images
processed 5001/6369 images
processed 5501/6369 images
processed 6001/6369 images


## Save Results

In [ ]:
BDD_API_results = {}
for img, result in API_results.items():
    BDD_API_results[img] = {}
    for key, value in result.items():
        if key != "text":
            BDD_API_results[img][key] = value

In [ ]:
with open(join(basedir, "BDD_API_result.json"), 'w') as f:
      json.dump(BDD_API_results, f)

In [ ]:
[text.description for text in BDD_API_results['0000f77c-6257be58.jpg']["texts"]]

['R Parking\nar Returns\nerminals\nNE WAY\n1139243\n',
 'R',
 'Parking',
 'ar',
 'Returns',
 'erminals',
 'NE',
 'WAY',
 '1139243']

In [ ]:
BDD_API_results["adfc8f5c-abf229da.jpg"]

{'landmarks': None,
 'logos': None,
 'texts': [['DURT\nBattery Tuncl\nBrouklyn\nMLE\n',
   [[155, 117], [1266, 117], [1266, 262], [155, 262]]],
  ['DURT', [[156, 238], [192, 239], [191, 262], [155, 261]]],
  ['Battery', [[1162, 143], [1213, 131], [1215, 143], [1165, 155]]],
  ['Tuncl', [[1221, 127], [1262, 117], [1266, 133], [1225, 143]]],
  ['Brouklyn', [[1175, 157], [1233, 145], [1235, 157], [1177, 169]]],
  ['MLE', [[1205, 194], [1222, 191], [1223, 199], [1206, 202]]]]}